# Debug notebook
This notebook is only used to troubleshoot the S1 ARD workflow processor.

## S1 ARD workflow for scenes overlapping AM

In [1]:
import timeout_decorator
import json
from utils.prepS1AM import prepareS1AM

@timeout_decorator.timeout(10800, use_signals=False)
def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    logging.info("%s: %s" % (x, loaded_json[x]))
    prepareS1AM(**loaded_json)

## S1 ARD job queue processor

In [2]:
import os
import logging
import rediswq

level = os.getenv("LOGLEVEL", "INFO").upper()
logging.basicConfig(format="%(asctime)s %(levelname)-8s %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=level)

host = os.getenv("REDIS_SERVICE_HOST", "redis-master")
q = rediswq.RedisWQ(name="jobS1AM", host=host)

logging.info("Worker with sessionID: " +  q.sessionID())
logging.info("Initial queue state: empty=" + str(q.empty()))

2020-10-16 10:54:59 INFO     Worker with sessionID: 7f4443d9-f854-4970-b96d-15d520268102
2020-10-16 10:54:59 INFO     Initial queue state: empty=False


In [ ]:
while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        logging.info("Working on " + itemstr)
        #time.sleep(10) # Put your actual work here instead of sleep.
        try:
            process_scene(itemstr)
        except timeout_decorator.TimeoutError:
            logging.info("Timed out while working on " + itemstr)
            pass
        q.complete(item)
    else:
        logging.info("Waiting for work")

2020-10-16 10:56:32 INFO     Working on {"in_scene": "S1A_IW_GRDH_1SDV_20201007T173247_20201007T173304_034693_040A80_8403", "s3_bucket": "public-eo-data", "s3_dir": "luigi/fiji/sentinel_1/"}
2020-10-16 10:56:32 INFO     S1A_IW_GRDH_1SDV_20201007T173247_20201007T173304_034693_040A80_8403.SAFE S1A_IW_GRDH_1SDV_20201007T173247 Starting
2020-10-16 10:56:32 INFO     S1A_IW_GRDH_1SDV_20201007T173247_20201007T173304_034693_040A80_8403.SAFE S1A_IW_GRDH_1SDV_20201007T173247 DOWNLOADING via ASF
2020-10-16 10:56:32 INFO     fetching: https://api.daac.asf.alaska.edu/services/search/param?granule_list=S1A_IW_GRDH_1SDV_20201007T173247_20201007T173304_034693_040A80_8403&output=csv
2020-10-16 10:56:33 INFO     logging into asf
2020-10-16 10:56:37 INFO     successfully logged into asf
2020-10-16 10:57:10 INFO     S1A_IW_GRDH_1SDV_20201007T173247_20201007T173304_034693_040A80_8403.SAFE S1A_IW_GRDH_1SDV_20201007T173247 DOWNLOADED via ASF
2020-10-16 10:57:10 INFO     S1A_IW_GRDH_1SDV_20201007T173247_20201

In [ ]:
print("Queue empty!")

---